<h1> Import libraries here </h1>

In [27]:
import sklearn as sk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model

## For Data visualisation 
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sn
# from adspy_shared_utilities import plot_class_regions_for_classifier_subplot #Save to git first


<h2>1. DATA PROCESSING</h2>

<h3>Functions</h3>

In [28]:
def split(x, y, train = 0.8, test = 0): 
    valid = 1 - train - test
    x_trainy, x_valid, y_trainy, y_valid = train_test_split(x, y, train_size = (train+test), test_size = valid)
    if test > 0:
        x_train, x_test, y_train, y_test = (
            train_test_split(x_trainy, y_trainy, train_size = train/(train+test), test_size = test/(train+test)))
    else:
        x_train, y_train =  x_trainy, y_trainy
        x_train, y_train =  x_trainy, y_trainy
        x_test, y_test = 0, 0
    return x_train, x_valid, x_test, y_train, y_valid, y_test

def score_RandomForest(x_train, x_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100)
    model.fit(x_train, y_train)
    preds = model.predict(x_valid)
    return model, mean_absolute_error(y_valid, preds)

## Keep getting different variables with the highest importance??
def LinReg (X, y, function):
    feature_list, importances, feature_importances = function
    sorted_by_second = sorted(feature_importances, key=lambda tup: tup[1], reverse=True)
    ## Edit ind_var to only use indepent variables 
    ind_var = ['Operator', 'Water depth', 'a1', 'Turbine rating (kW)', 'Blade length (m)', 'Tower height (m)', 'Built duration', 'Type', 'Metocean', 'Region']
    X = X[ind_var]
    regr = linear_model.LinearRegression()
    regr.fit(X,y)
    pass

def feature_importance(model, x_train):
    feature_list = list(x_train.columns)
    importances = model.feature_importances_
    feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
    # Filter importance > 0.1 only
    feature_importances = [item for item in feature_importances if item[1] > 0.1]
    return feature_list, importances, feature_importances


<h3> Preprocessing </h3>

In [29]:
dataPure = pd.read_excel("Digital_Data.xlsx")
data = dataPure.copy()
data = data.replace(to_replace = 0, value = np.NaN)
label_encoder = LabelEncoder()
s = (data.dtypes == 'object')
cat_cols = list(s[s].index)
for col in cat_cols:
    data[col] = label_encoder.fit_transform(data[col].astype(str))

##Dropping Features HERE
data = data.drop(["Project Ref No.", "Blade length (m)", "Tower height (m)","Built duration"], axis = 1)

data_sets = [data.drop(["Single Blade Weight (te)"], axis = 1), data.drop(["Nacelle Weights"], axis = 1)]
X = []
Y = []
for i in range(2):
    data_sets[i] = data_sets[i].dropna()
    x = data_sets[i].drop(columns = data_sets[i].columns[4])
    y = data_sets[i][data_sets[i].columns[4]]
    x_train, x_valid, x_test, y_train, y_valid, y_test = split(x,y)
    X.append([x_train, x_valid, x_test])
    Y.append([y_train, y_valid, y_test])

<h3>Random Forrest</h3>

In [30]:
model = []
for i in range(2):
    m, mae = score_RandomForest(X[i][0], X[i][1], Y[i][0], Y[i][1])
    model.append(m)
    print(mae)

24.02532
3.4602083333333336


<h3>Lin Regression and Independent Variables</h3>


In [31]:
feature_import = []
for i in range(2):
    ## Identify the features of importance from Random Forest
    feature_import.append(feature_importance(model[i], X[i][0]))
    ## Multivariable Linear Regression
    # LinReg(x, y, feature_import)
print(feature_import[0][2])
print('\n')
print(feature_import[1][2])

[('a1', 0.48), ('Turbine rating (kW)', 0.26)]


[('Water depth', 0.2), ('a1', 0.29), ('Turbine rating (kW)', 0.24), ('Region', 0.12)]


In [32]:
[('Operator', 0.04), ('Water depth', 0.04), ('a1', 0.29), ('Turbine rating (kW)', 0.41), ('Type', 0.13), ('Metocean', 0.01), ('Region', 0.08)]


[('Operator', 0.0), ('Water depth', 0.34), ('a1', 0.27), ('Turbine rating (kW)', 0.15), ('Type', 0.08), ('Metocean', 0.04), ('Region', 0.11)]

[('Operator', 0.0),
 ('Water depth', 0.34),
 ('a1', 0.27),
 ('Turbine rating (kW)', 0.15),
 ('Type', 0.08),
 ('Metocean', 0.04),
 ('Region', 0.11)]